# Blast Conservation

In [2]:
with open('../fasta_files/NeurD2_aaseq.fa') as f:
    seq = f.read()
print(seq)

>sp|Q15784|NDF2_HUMAN Neurogenic differentiation factor 2 OS=Homo sapiens OX=9606 GN=NEUROD2 PE=1 SV=2
MLTRLFSEPGLLSDVPKFASWGDGEDDEPRSDKGDAPPPPPPAPGPGAPGPARAAKPVPL
RGEEGTEATLAEVKEEGELGGEEEEEEEEEEGLDEAEGERPKKRGPKKRKMTKARLERSK
LRRQKANARERNRMHDLNAALDNLRKVVPCYSKTQKLSKIETLRLAKNYIWALSEILRSG
KRPDLVSYVQTLCKGLSQPTTNLVAGCLQLNSRNFLTEQGADGAGRFHGSGGPFAMHPYP
YPCSRLAGAQCQAAGGLGGGAAHALRTHGYCAAYETLYAAAGGGGASPDYNSSEYEGPLS
PPLCLNGNFSLKQDSSPDHEKSYHYSMHYSALPGSRPTGHGLVFGSSAVRGGVHSENLLS
YDMHLHHDRGPMYEELNAFFHN



## 1. Run BLAST
```bash
blastp -query NeurD2_aaseq.fa -db ../ExerciseBLAST/databases/swissprot -out myseq_swissprot.out
```

# Phylogenetic Tree with Sequence Conservation Visualization

## Homologs

### Import libraries and Define File Paths

In [ ]:
from ete4 import Tree
from ete4.treeview import TreeStyle, SequenceFace
from Bio import SeqIO

output_figures = "../figures/Phylogenetic_trees/"
MAIN_GENE = "sp|Q15784|NDF2_HUMAN"   
TREE_FILE = "../output_files/FastTree/Orthologs_Tree.nwk"
MSA_FILE  = "../output_files/MSA/orthologs_clipkit.fa"


## Normal Tree

In [4]:
t = Tree("../output_files/FastTree/Orthologs_Tree.nwk")
circular_style = TreeStyle()    
circular_style.show_leaf_name = True
circular_style.mode = "c" # draw tree in circular mode
circular_style.scale = 60
circular_style.branch_vertical_margin = 10
circular_style.show_leaf_name = True
for leaf in t.iter_leaves():
    leaf_style = leaf._get_style()
    leaf_style["size"] = 10           # node size
    leaf_style["fgcolor"] = "black"   # label color
    leaf.set_style(leaf_style)
t.render("../figures/Phylogenetic_trees/Homologs_circular_tree.png",  w=4000, units="px", tree_style=circular_style, dpi=300)
print("Tree rendered and saved as mytree.png")


Tree rendered and saved as mytree.png


## Full tree with MSA sequences

In [ ]:
seqs = {}
for rec in SeqIO.parse("../output_files/MSA/orthologs_clipkit.fa", "fasta"):
    seqs[rec.id] = str(rec.seq)

# Load tree
t = Tree("../output_files/FastTree/Orthologs_Tree.nwk")

for leaf in t.leaves():
    if leaf.name in seqs:
        seq = seqs[leaf.name]

        # BLOCKSEQ
        face1 = SequenceFace(seq, seqtype="aa")
        leaf.add_face(face1, column=0, position="aligned")

        # COMPACTSEQ
        face2 = SequenceFace(seq, seqtype="aa")
        leaf.add_face(face2, column=1, position="aligned")

        # FULLSEQ
        face3 = SequenceFace(seq, seqtype="aa")
        leaf.add_face(face3, column=2, position="aligned")

ts = TreeStyle()
ts.mode = "c"
ts.show_leaf_name = True
ts.scale = 100

t.render(output_figures + "tree_with_sequences.png", w=12000, units="px", dpi=600, tree_style=ts)


qt.qpa.fonts: Populating font family aliases took 43 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 


Wrote file: ../figures/Phylogenetic_trees/tree_with_sequences.png


## Closest sequences subtree with MSA sequences

In [ ]:
OUTPUT_FILE = output_figures + "NEUROD2_closest_tree.png"

t = Tree(TREE_FILE)

seqs = {rec.id: str(rec.seq) for rec in SeqIO.parse(MSA_FILE, "fasta")}

try:
    main = next(t.search_leaves_by_name(MAIN_GENE))
except StopIteration:
    raise ValueError(f"Gene {MAIN_GENE} not found in the tree!")


distances = [(leaf, leaf.get_distance(leaf,main)) for leaf in t.leaves()]

closest_names = [leaf.name for leaf, _ in sorted(distances, key=lambda x: x[1])[:N_CLOSEST]]

subtree = t.copy()
subtree.prune(closest_names)

for leaf in subtree.leaves():
    if leaf.name not in seqs:
        continue

    seq = seqs[leaf.name].replace("U", "C")  

    leaf.add_face(SequenceFace(seq, seqtype="aa"), column=0, position="aligned")

    leaf.add_face(SequenceFace(seq, seqtype="aa"), column=1, position="aligned")

    leaf.add_face(SequenceFace(seq, seqtype="aa"), column=2, position="aligned")

ts = TreeStyle()
ts.mode = "c"
ts.show_leaf_name = True
ts.scale = 60

# Title (use TextFace instead of SequenceFace)
from ete4.treeview import TextFace
ts.title.add_face(TextFace("NEUROD2 Closest Sequences", fsize=20, fgcolor="black"), column=0)

# Render high-resolution image
subtree.render(
    OUTPUT_FILE,
    w=12000,
    units="px",
    dpi=600,
    tree_style=ts
)

print(f"DONE: {OUTPUT_FILE} created!")

Wrote file: NEUROD2_closest_tree.png
DONE: NEUROD2_closest_tree.png created!


## Rectangular closest subtree with MSA sequences

In [12]:
OUTPUT_FILE = output_figures + "NEUROD2_subtree.png"
N_CLOSEST = 30                     

t = Tree(TREE_FILE)

seqs = {rec.id: str(rec.seq).replace("U", "C") for rec in SeqIO.parse(MSA_FILE, "fasta")}

main = next(t.search_leaves_by_name(MAIN_GENE))

distances = [(leaf, t.get_distance(leaf, main)) for leaf in t.leaves()]

closest_names = [leaf.name for leaf, _ in sorted(distances, key=lambda x: x[1]) 
                 if leaf.name in seqs][:N_CLOSEST]

subtree = t.copy()
subtree.prune(closest_names)

for leaf in subtree.leaves():
    seq = seqs.get(leaf.name, "")
    if not seq:
        continue
    leaf.add_face(SequenceFace(seq, seqtype="aa"), column=0, position="aligned")

ts = TreeStyle()
ts.show_leaf_name = True
ts.scale = 50

subtree.render(
    OUTPUT_FILE,
    w=15000,
    units="px",
    dpi=600,
    tree_style=ts
)


Wrote file: ../figures/Phylogenetic_trees/NEUROD2_subtree.png


# Closest Homologs with Images

In [ ]:
from ete4 import Tree
from ete4.treeview import TreeStyle, SequenceFace, TextFace, ImgFace, faces
from Bio import SeqIO


MAIN_GENE = "sp|Q15784|NDF2_HUMAN"
N_CLOSEST = 10
TREE_FILE = "../output_files/FastTree/Orthologs_Tree.nwk"
MSA_FILE  = "../output_files/MSA/orthologs_clipkit.fa"
OUTPUT_FILE = "NEUROD2_closest10.png"

t = Tree(TREE_FILE)


seqs = {rec.id: str(rec.seq).replace("U", "C") for rec in SeqIO.parse(MSA_FILE, "fasta")}


main = next(t.search_leaves_by_name(MAIN_GENE))


distances = [(leaf, t.get_distance(leaf, main)) for leaf in t.leaves()]

closest_names = [leaf.name for leaf, dist in sorted(distances, key=lambda x: x[1])
                 if leaf.name in seqs][:N_CLOSEST]


subtree = t.copy()
subtree.prune(closest_names)



def mylayout(node):
    if node.is_leaf:   
        nameFace = faces.TextFace(node.name, fsize=12, fgcolor="darkgreen")
        faces.add_face_to_node(nameFace, node, column=0, position="aligned")
        # Optional: add sequence length
        seq = seqs.get(node.name, "")
        if seq:
            seqFace = faces.TextFace(f"Length: {len(seq)}", fsize=10, fgcolor="blue")
            faces.add_face_to_node(seqFace, node, column=1, position="aligned")
        node.img_style["shape"] = "circle"
        node.img_style["size"] = 10
    else:
        node.img_style["shape"] = "sphere"
        node.img_style["size"] = 6
        node.img_style["fgcolor"] = "black"

ts = TreeStyle()
ts.layout_fn = mylayout
ts.show_leaf_name = False
ts.scale = 50


subtree.render(
    OUTPUT_FILE,
    w=12000,
    units="px",
    dpi=300,
    tree_style=ts
)



Wrote file: NEUROD2_closest10.png
Done! Subtree saved as NEUROD2_closest10.png


In [17]:
closest_names

['sp|Q15784|NDF2_HUMAN',
 'sp|Q62414|NDF2_MOUSE',
 'sp|Q63689|NDF2_RAT',
 'sp|Q9W6C8|NDF2_DANRE',
 'sp|Q60867|NDF1_MOUSE',
 'sp|Q64289|NDF1_RAT',
 'sp|Q60430|NDF1_MESAU',
 'sp|Q4R5G6|NDF6_MACFA',
 'sp|Q08DI0|NDF6_BOVIN',
 'sp|P48986|NDF6_MOUSE']

In [52]:
from ete4 import Tree
from ete4.treeview import TreeStyle, TextFace, ImgFace, faces
from Bio import SeqIO
import os

# -----------------------------
# 1. Settings
# -----------------------------
MAIN_GENE = "sp|Q15784|NDF2_HUMAN"
N_CLOSEST = 10
TREE_FILE = "../output_files/FastTree/Orthologs_Tree.nwk"
MSA_FILE  = "../output_files/MSA/orthologs_clipkit.fa"
OUTPUT_FILE =  output_figures+"NEUROD2_closest10.png"

# -----------------------------
# 2. Load tree and sequences
# -----------------------------
t = Tree(TREE_FILE)
seqs = {rec.id: str(rec.seq).replace("U", "C") for rec in SeqIO.parse(MSA_FILE, "fasta")}

# -----------------------------
# 3. Find main gene leaf
# -----------------------------
main = next(t.search_leaves_by_name(MAIN_GENE))

# -----------------------------
# 4. Compute distances to all leaves
# -----------------------------
distances = [(leaf, t.get_distance(leaf, main)) for leaf in t.leaves()]

# Get N closest leaves that also have sequences
closest_names = [leaf.name for leaf, dist in sorted(distances, key=lambda x: x[1])
                 if leaf.name in seqs][:N_CLOSEST]

# -----------------------------
# 5. Prune tree to closest leaves
# -----------------------------
subtree = t.copy()
subtree.prune(closest_names)

# -----------------------------
# 6. Paths to images (example)
# -----------------------------
img_path = "../figures/related_organisms/"  # folder containing your images
image_faces = {name: ImgFace(img_path + f"{name}.jpg") for name in closest_names}  # placeholder

# -----------------------------
# 7. Optional labels / descriptions
# -----------------------------
code2name = {name: f"Species {name}" for name in closest_names}     # replace with real names
code2desc = {name: f"Description of {name}" for name in closest_names}  # replace with real descriptions

# -----------------------------
# 8. Custom layout function
# -----------------------------
def mylayout(node):
    if node.is_leaf:
        # Leaf name
        faces.add_face_to_node(TextFace(node.name, fsize=12, fgcolor="darkgreen"), node, column=0, position="aligned")
        # Scientific/common name
        faces.add_face_to_node(TextFace(code2name.get(node.name, ""), fsize=10, fgcolor="blue"), node, column=1, position="aligned")
        # Description
        desc_face = TextFace(code2desc.get(node.name, ""), fsize=8)
        desc_face.margin_top = 5
        desc_face.margin_bottom = 5
        faces.add_face_to_node(desc_face, node, column=2, position="aligned")
        # Image face if available
        if node.name in image_faces:
            img_file = img_path + f"{node.name}.jpg"
            if os.path.exists(img_file):
                faces.add_face_to_node(ImgFace(img_file, width=1920, height=1080), node, column=3)
        # Leaf style
        node.img_style["shape"] = "circle"
        node.img_style["size"] = 10
    else:
        # Internal node style
        node.img_style["shape"] = "sphere"
        node.img_style["size"] = 10
        node.img_style["fgcolor"] = "black"

# -----------------------------
# 9. TreeStyle
# -----------------------------
ts = TreeStyle()
ts.layout_fn = mylayout
ts.show_leaf_name = False
ts.scale = 50

# -----------------------------
# 10. Render
# -----------------------------
subtree.render(OUTPUT_FILE, w=12000, tree_style=ts, dpi=600, units="px")
print(f"Done! Subtree saved as {OUTPUT_FILE}")

Wrote file: ../figures/Phylogenetic_trees/NEUROD2_closest10.png
Done! Subtree saved as ../figures/Phylogenetic_trees/NEUROD2_closest10.png
